In [1]:
import os
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient, BlobBlock
import pandas as pd
import io
import csv
import re

In [2]:
account_name='datalake2dictas'
account_key='wlxJHcWvtVhPpL/zs6l+F1bJGKZnJ4HppZcVyh+ns32oH46E3dY/HBLau3V6um9hv+KZf/3mXEAL5nHD41X3jg=='
container_from = 'transient'
container_to = 'raw'
upload_file = '/opt/jupyter_etl_dictas/datalake/transient/beneficiario/beneficiarios_atualizado.csv'
_LOCALDIR = '/opt/jupyter_etl_dictas/datalake/transient'

In [3]:
cc_from = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_from, credential=account_key)
cc_to = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_to, credential=account_key)
cc_trusted = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name='trusted', credential=account_key)

In [13]:
for x in cc_trusted.list_blobs ():
    z = cc_trusted.get_blob_client(x)
    z.delete_blob()

In [20]:
for x in cc_from.list_blobs ():
    z = cc_from.get_blob_client(x)
    z.delete_blob()

for x in cc_to.list_blobs ():
    z = cc_to.get_blob_client(x)
    z.delete_blob()

for x in cc_trusted.list_blobs ():
    z = cc_trusted.get_blob_client(x)
    z.delete_blob()    

In [4]:
ldatalake_from = [file['name'].replace('.csv','') for file in cc_from.list_blobs ()]
ldatalake_to = [file['name'].replace('.pickle','') for file in cc_to.list_blobs ()]
ldatalake_trusted = [file['name'] for file in cc_trusted.list_blobs ()]

In [5]:
ldatalake_trusted

['beneficiario/BENEFICIARIOS.pkl',
 'beneficiario/BENEFICIARIOS_error.pkl',
 'consultas/CONSULTAS.pkl',
 'custo/CUSTO 012018.pkl',
 'custo/CUSTO 012018_error.pkl',
 'custo/CUSTO 012019.pkl',
 'custo/CUSTO 012019_error.pkl',
 'custo/CUSTO 022017.pkl',
 'custo/CUSTO 022017_error.pkl',
 'custo/CUSTO 022018.pkl',
 'custo/CUSTO 022018_error.pkl',
 'custo/CUSTO 022019.pkl',
 'custo/CUSTO 022019_error.pkl',
 'custo/CUSTO 032017.pkl',
 'custo/CUSTO 032017_error.pkl',
 'custo/CUSTO 032018.pkl',
 'custo/CUSTO 032018_error.pkl',
 'custo/CUSTO 032019.pkl',
 'custo/CUSTO 032019_error.pkl',
 'custo/CUSTO 042017.pkl',
 'custo/CUSTO 042017_error.pkl',
 'custo/CUSTO 042018.pkl',
 'custo/CUSTO 042018_error.pkl',
 'custo/CUSTO 042019.pkl',
 'custo/CUSTO 042019_error.pkl',
 'custo/CUSTO 052017.pkl',
 'custo/CUSTO 052017_error.pkl',
 'custo/CUSTO 052018.pkl',
 'custo/CUSTO 052018_error.pkl',
 'custo/CUSTO 052019.pkl',
 'custo/CUSTO 052019_error.pkl',
 'custo/CUSTO 062017.pkl',
 'custo/CUSTO 062017_error.pk

In [ ]:
print(len(ldatalake_from), len(ldatalake_to), len(ldatalake_trusted))

In [ ]:
['beneficiario/BENEFICIARIOS','consultas/CONSULTAS','diarias/DIARIAS','empresa/EMPRESA','prestador/PRESTADORES','vidas/VIDAS']

In [ ]:
ldatalake_from

In [ ]:
bc = cc_to.get_blob_client('custo/CUSTO 122018.pickle')

In [ ]:
%%time
with open('/opt/jupyter_etl_dictas/notebook/CUSTO 122018.pickle','wb') as data_from:
    data_from.write(bc.download_blob().readall())    

In [ ]:
df = pd.read_pickle('/opt/jupyter_etl_dictas/notebook/CUSTO 122018.pickle')

In [ ]:
df.head()

In [ ]:
df.head(1).T

In [ ]:
def replacevalue(df, val, replace_val) :
    for col in df.columns :
        df[col] = df[col].apply (lambda x :re.sub (val, replace_val, str (x)))
    return df

In [ ]:
df2 = df.copy()

In [ ]:
df2 = replacevalue(df2,r'nan|NULL|[\t]','')
df2 = df2.apply (lambda x :x.str.strip ().replace ('', None))

In [ ]:
df2['nr_guia_principal'].apply(lambda x: '-1' if x=='' else x).astype(int)

In [ ]:
df2[df2['nr_guia_principal']=='VT']

In [ ]:
cols_integer = ['Cod. Beneficiário', 'Nr. Carteira', 'Nr. Carteira Titular', 
            'Cod. Empresa', 'CPF', 'Cod. Plano']

In [ ]:
df2[cols_integer].apply(lambda x : x.str.strip().replace('',None)).astype(int)

In [ ]:
def transform_to_datetime(dt_in_string, date_format='%d/%m/%Y'):
    if dt_in_string == '':
        return None
    else:
        try:
            df_in_datime = pd.to_datetime(dt_in_string, format=date_format)
            return df_in_datime
        except pd.errors.OutOfBoundsDatetime:
            return None

In [ ]:
df2[['Dt. Nascimento','Dt. Inclusão']].apply(pd.to_datetime,format='%d/%m/%Y', errors='coerce')

In [ ]:
df2.head(30).T.reset_index()

In [ ]:
def sel_no_int_cols(cols):
    validatecols = []
    for col in cols:
        try:
            int(col)
        except:
            validatecols.append(col)
    return validatecols        

In [ ]:
df2 = df2[selectnointegercol(df2.columns)]

In [ ]:
df2.dtypes

In [ ]:
%%time
dblob = bc.download_blob().readall()

In [ ]:
bc.download_blob().readinto(stream=)

In [ ]:
type(dblob)

In [ ]:
df = pd.read_pickle(io.BytesIO(dblob))

In [ ]:
len(list_c)

In [ ]:
len(ldatalake_from)/3

In [ ]:
len(ldatalake_from)

In [ ]:
len(ldatalake_to)

In [ ]:
bc = cc_to.get_blob_client('DIARIAS.csv')
bc.delete_blob()

In [ ]:
bc = cc_from.get_blob_client('receitas/RECEITA 032017.csv')

In [ ]:
%%time
dblob = bc.download_blob().readall()

In [ ]:
df = pd.read_csv(io.BytesIO(dblob),sep="|", 
                 error_bad_lines=False,encoding='utf8' , 
                 quoting=csv.QUOTE_NONE,
                 low_memory=False,
                 header=None)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
prestador = ['Cod. Prestador', 'Nome', 'CRM', 'UF CRM', 'CPF/CNPJ', 'Tipo', 'Grupo', 'Descrição da Especialidade',
    'Especialidade Principal']

receitas = ['Mês Competência', 'Cod. Beneficiário', 'Cod. Empresa', 'Dt. Geração Título', 'Tipo Cobrança','Vl. Cobrança', 'Vl. Pago']

lcol = [x for x in df.columns if x < len(receitas)]
dict_prest = dict(zip(lcol,prestador))
df.rename(columns=dict_prest)

In [ ]:
df.shape

In [ ]:
df.rename(columns=dict_prest)

In [ ]:
len(prestador)

In [ ]:
lcol = [x for x in df.columns if x < len(prestador)]

In [ ]:
dict(zip(lcol,prestador))

In [ ]:
bc.delete_blob()

In [ ]:
bc.upload_blob()

In [ ]:
ldatalake = [f"{file['name']}" for file in cc.list_blobs()]
len(ldatalake)

In [ ]:
ldatalake

In [ ]:
llocal = [os.path.join(currentpath, file).replace(_LOCALDIR,'')[1:] 
 for currentpath, folders, files in os.walk(_LOCALDIR) 
 for file in files]
len(llocal)

In [ ]:
ldatalake = [f"{file['name']}" for file in cc.list_blobs()]

In [ ]:
ldatalake = [f"{file['name']}" for file in cc.list_blobs()]

llocal = [os.path.join(currentpath, file).replace(_LOCALDIR,'')[1:] 
 for currentpath, folders, files in os.walk(_LOCALDIR) 
 for file in files]

ldiff = [file for file in llocal if file not in ldatalake]

In [ ]:
'custos_competencia/CUSTO 042018.csv',
'custos_atualizado/CUSTO 042018.csv',

In [ ]:
ldiff

In [ ]:
ldatalake

In [ ]:
with open(f'{_LOCALDIR}/diff.txt','r') as diff:
    a = [x for x in diff.read().split('\n') if len(x)>0]

In [ ]:
for x in a:
    print(x.split('/')[2])

In [13]:
l_test = []
a = 'aa'
b = 'bb'
c='cc'
d=None

In [9]:
l_test.append(a)

In [11]:
l_test.append(b)

In [14]:
l_test.append(d)

In [15]:
l_test

[None]